# Downloading Images and Collections
In this tutorial, we'll look at how we can use `wxee` and the `wx` accesor to quickly and easily download images and image collections from Earth Engine to a local drive. 

`wxee` isn't designed to replace Earth Engine's native downloading functionality--if you need high resolution images or large areas, downloading to a Drive using ee is probably the best option. What `wxee`'s downloading system is great for is quickly pulling small regions or mesoscale images for further processing in Python without the hassle of intermediate steps.

## Setup

In [ ]:
!pip install wxee

In [1]:
import ee
import wxee

ee.Authenticate()
ee.Initialize()

## Downloading an Image

Load an image from MODIS. For convenience, we'll select just the red, green, and blue bands.

In [2]:
img = ee.Image("MODIS/006/MOD09GA/2021_06_01").select(["sur_refl_b01", "sur_refl_b04" ,"sur_refl_b03"])

Set the download parameters.

In [3]:
# The file name to save
description = "modis_img"
# The coordinate reference system to use (NAD83 Albers CONUS)
crs = "EPSG:5070"
# Spatial resolution in CRS units (meters)
scale = 500
# The region to download the image within.
region = ee.Geometry.Polygon(
    [[[-120.0576549852371, 46.185091976777684],
      [-120.0576549852371, 45.577074504710005],
      [-118.91782344226834, 45.577074504710005],
      [-118.91782344226834, 46.185091976777684]]]
)

<div class="alert alert-block alert-warning">
<b>Warning:</b> If an image download fails due to size limits, try using a smaller region or a coarser scale.
</div>

Download the image to GeoTIFF. The `ee.Image.wx.to_tif` method returns a path to the downloaded image(s). `wxee` always saves files with the name format `{id}.{dimension}.{coordinate}`. If no `description` is passed, the `system:id` property of the image will determine the file ID.

In [4]:
file = img.wx.to_tif(out_dir='data', description=description, region=region, scale=scale, crs=crs)
file

Downloading:   0%|          | 0.00/139k [00:00<?, ?iB/s]

['data/modis_img.time.20210601T000000.tif']

Above, we downloaded 1 three-band image, but we could also download it as 3 one-band images by setting `file_per_band` to True.

In [5]:
file = img.wx.to_tif(
    out_dir='data', 
    description=description, 
    region=region, 
    scale=scale, 
    crs=crs, 
    file_per_band=True
)

file

Downloading:   0%|          | 0.00/159k [00:00<?, ?iB/s]

['data/modis_img.time.20210601T000000.sur_refl_b01.tif',
 'data/modis_img.time.20210601T000000.sur_refl_b04.tif',
 'data/modis_img.time.20210601T000000.sur_refl_b03.tif']

## Downloading an Image Collection

Load the MODIS collection, choose a date range with 4 images, and select the red, green, and blue bands.

In [6]:
collection = (ee.ImageCollection("MODIS/006/MOD09GA")
    .filterDate("2021-06-01", "2021-06-05")
    .select(["sur_refl_b01", "sur_refl_b04" ,"sur_refl_b03"])
)

Download each image in the collection to GeoTIFF. We'll use the same parameters as above except for `description`. `ee.ImageCollection.wx.to_tif` doesn't take a `description` parameter, but it does take a `prefix` that can be added to the beginning of each file name. File names are based on the `system:id` property of each image.

In [7]:
files = collection.wx.to_tif(
    out_dir="data", 
    prefix="wx_", 
    region=region, 
    scale=scale, 
    crs=crs
)

files

Requesting:   0%|          | 0/4 [00:00<?, ?it/s]

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]

['data/wx_MODIS_006_MOD09GA_2021_06_02.time.20210602T000000.tif',
 'data/wx_MODIS_006_MOD09GA_2021_06_01.time.20210601T000000.tif',
 'data/wx_MODIS_006_MOD09GA_2021_06_04.time.20210604T000000.tif',
 'data/wx_MODIS_006_MOD09GA_2021_06_03.time.20210603T000000.tif']